In [5]:
def cal_metrics(csv_path, is_binary=True, num=2):
    '''
    Calculate average accuracy, accuracy per skin type, PQD, DPM, EOM.
    All known skin types.
    Input: val results csv path, type_indices: a list
    Output: a dictionary with 'acc_avg', 'acc_per_type', 'PQD', 'DPM', 'EOM'
    '''
    import numpy as np
    import pandas as pd
    from sklearn.metrics import roc_curve, auc

    df = pd.read_csv(csv_path)

    # Calculate the number of groups based on Fitzpatrick types
    num_groups = (6 - 1) // num + 1

    # Create arrays to store the results
    labels_array = np.zeros((num_groups, len(df['label'].unique())))
    correct_array = np.zeros((num_groups, len(df['label'].unique())))
    predictions_array = np.zeros((num_groups, len(df['label'].unique())))
    positive_list = []  # For binary classification probabilities
    
    for i in range(df.shape[0]):
        prediction = df.iloc[i]['prediction']
        label = df.iloc[i]['label']
        type_ = (df.iloc[i]['fitzpatrick'] - 1) // num
        labels_array[int(type_), int(label)] += 1
        predictions_array[int(type_), int(prediction)] += 1
        if prediction == label:
            correct_array[int(type_), int(label)] += 1

        if is_binary:
            if prediction == 0:
                positive_list.append(1.0 - df.iloc[i]['prediction_probability'])
            else:
                positive_list.append(df.iloc[i]['prediction_probability'])
    
    # Avoid division by zero by adding checks and replacing zero values with small positive constants
    eps = 1e-6  # Small constant to avoid zero values
    
    correct_array_sumc = np.sum(correct_array, axis=1)
    labels_array_sumc = np.sum(labels_array, axis=1)
    acc_array = np.divide(correct_array_sumc, labels_array_sumc + eps)

    avg_acc = np.sum(correct_array) / (np.sum(labels_array) + eps)
    
    # PQD: Compute min/max accuracy safely
    acc_min = np.min(acc_array[acc_array > 0]) if np.any(acc_array > 0) else eps
    acc_max = np.max(acc_array) if np.max(acc_array) > 0 else eps
    PQD = acc_min / acc_max

    # DPM: Compute demo_array and ensure no zero division
    demo_array = np.divide(predictions_array, np.sum(predictions_array, axis=1, keepdims=True) + eps)
    demo_min = np.min(demo_array, axis=0)
    demo_max = np.max(demo_array, axis=0)
    DPM = np.mean(np.divide(demo_min, demo_max + eps))

    # EOM: Compute element-wise equality of opportunity
    eo_array = np.divide(correct_array, labels_array + eps)
    eo_min = np.min(eo_array, axis=0)
    eo_max = np.max(eo_array, axis=0)
    EOM = np.mean(np.divide(eo_min, eo_max + eps))

    # Calculate AUC for binary classification
    AUC = -1
    if is_binary:
        fpr, tpr, _ = roc_curve(df['label'], positive_list, drop_intermediate=True)
        AUC = auc(fpr, tpr)

    return {
        'acc_avg': avg_acc, 
        'acc_per_type': acc_array.tolist(),  # Convert numpy array to list for readability
        'PQD': PQD, 
        'DPM': DPM, 
        'EOM': EOM, 
        'AUC': AUC
    }


In [1]:
def cal_metrics(csv_path, is_binary=True, num=2):
    import numpy as np
    import pandas as pd
    from sklearn.metrics import roc_curve, auc

    df = pd.read_csv(csv_path)

    num_groups = (6 - 1) // num + 1

    labels_array = np.zeros((num_groups, len(df['label'].unique())))
    correct_array = np.zeros((num_groups, len(df['label'].unique())))
    predictions_array = np.zeros((num_groups, len(df['label'].unique())))
    acc_array = np.zeros(num_groups)

    group_probs = [[] for _ in range(num_groups)]
    group_labels = [[] for _ in range(num_groups)]

    positive_list = []
    global_labels = []

    for i in range(df.shape[0]):
        prediction = df.iloc[i]['prediction']
        label = df.iloc[i]['label']
        prob = df.iloc[i]['prediction_probability']
        type_ = (df.iloc[i]['fitzpatrick'] - 1) // num

        labels_array[type_, int(label)] += 1
        predictions_array[type_, int(prediction)] += 1
        if prediction == label:
            correct_array[type_, int(label)] += 1

        if is_binary:
            prob_pos = prob if prediction == 1 else 1.0 - prob
            positive_list.append(prob_pos)
            global_labels.append(label)

            group_probs[type_].append(prob)
            group_labels[type_].append(label)

    eps = 1e-6

    correct_array_sumc = np.sum(correct_array, axis=1)
    labels_array_sumc = np.sum(labels_array, axis=1)
    acc_array = np.divide(correct_array_sumc, labels_array_sumc + eps)

    avg_acc = np.sum(correct_array) / (np.sum(labels_array) + eps)

    acc_min = np.min(acc_array[acc_array > 0]) if np.any(acc_array > 0) else eps
    acc_max = np.max(acc_array) if np.max(acc_array) > 0 else eps
    PQD = acc_min / acc_max

    demo_array = np.divide(predictions_array, np.sum(predictions_array, axis=1, keepdims=True) + eps)
    demo_min = np.min(demo_array, axis=0)
    demo_max = np.max(demo_array, axis=0)
    DPM = np.mean(np.divide(demo_min, demo_max + eps))

    eo_array = np.divide(correct_array, labels_array + eps)
    eo_min = np.min(eo_array, axis=0)
    eo_max = np.max(eo_array, axis=0)
    EOM = np.mean(np.divide(eo_min, eo_max + eps))

    result = {
        'acc_avg': avg_acc, 
        'acc_per_type': acc_array.tolist(), 
        'PQD': PQD, 
        'DPM': DPM, 
        'EOM': EOM
    }

    if is_binary:
        # Global AUC
        fpr, tpr, _ = roc_curve(global_labels, positive_list, drop_intermediate=True)
        AUC = auc(fpr, tpr)
        result['AUC'] = AUC

        # Group-wise AUCs
        aucs = []
        tprs = []
        fprs = []
        for probs, labels in zip(group_probs, group_labels):
            if len(set(labels)) < 2:
                aucs.append(None)
                tprs.append(None)
                fprs.append(None)
                continue
            fpr_g, tpr_g, _ = roc_curve(labels, probs)
            auc_g = auc(fpr_g, tpr_g)
            aucs.append(auc_g)
            tprs.append(tpr_g[1])  # TPR at first threshold (TP / (TP + FN))
            fprs.append(fpr_g[1])  # FPR at first threshold (FP / (FP + TN))

        valid_aucs = [a for a in aucs if a is not None]
        result['min_auc'] = min(valid_aucs) if valid_aucs else None
        result['max_auc'] = max(valid_aucs) if valid_aucs else None

        # Equalized Odds Difference
        tprs_valid = [t for t in tprs if t is not None]
        fprs_valid = [f for f in fprs if f is not None]
        if tprs_valid and fprs_valid:
            eod = max(abs(max(tprs_valid) - min(tprs_valid)), abs(max(fprs_valid) - min(fprs_valid)))
        else:
            eod = None
        result['EOD'] = eod

    return result


In [1]:
#binary sensitive attribute value 
def cal_metrics(csv_path, is_binary=True):
    import numpy as np
    import pandas as pd
    from sklearn.metrics import roc_curve, auc

    df = pd.read_csv(csv_path)

    num_groups = 2  # Fixed since we only have Fitzpatrick types 1 and 2

    label_classes = sorted(df['label'].unique())
    num_classes = len(label_classes)

    labels_array = np.zeros((num_groups, num_classes))
    correct_array = np.zeros((num_groups, num_classes))
    predictions_array = np.zeros((num_groups, num_classes))
    acc_array = np.zeros(num_groups)

    group_probs = [[] for _ in range(num_groups)]
    group_labels = [[] for _ in range(num_groups)]

    positive_list = []
    global_labels = []

    for i in range(df.shape[0]):
        prediction = df.iloc[i]['prediction']
        label = df.iloc[i]['label']
        prob = df.iloc[i]['prediction_probability']
        fitz = int(df.iloc[i]['fitzpatrick'])
        type_ = fitz - 1  # Since fitzpatrick values are 1 or 2 → type_ is 0 or 1

        labels_array[type_, int(label)] += 1
        predictions_array[type_, int(prediction)] += 1
        if prediction == label:
            correct_array[type_, int(label)] += 1

        if is_binary:
            prob_pos = prob if prediction == 1 else 1.0 - prob
            positive_list.append(prob_pos)
            global_labels.append(label)

            group_probs[type_].append(prob)
            group_labels[type_].append(label)

    eps = 1e-6
    acc_array = np.sum(correct_array, axis=1) / (np.sum(labels_array, axis=1) + eps)
    avg_acc = np.sum(correct_array) / (np.sum(labels_array) + eps)

    acc_min = np.min(acc_array[acc_array > 0]) if np.any(acc_array > 0) else eps
    acc_max = np.max(acc_array) if np.max(acc_array) > 0 else eps
    PQD = acc_min / acc_max

    demo_array = np.divide(predictions_array, np.sum(predictions_array, axis=1, keepdims=True) + eps)
    demo_min = np.min(demo_array, axis=0)
    demo_max = np.max(demo_array, axis=0)
    DPM = np.mean(np.divide(demo_min, demo_max + eps))

    eo_array = np.divide(correct_array, labels_array + eps)
    eo_min = np.min(eo_array, axis=0)
    eo_max = np.max(eo_array, axis=0)
    EOM = np.mean(np.divide(eo_min, eo_max + eps))

    result = {
        'acc_avg': avg_acc,
        'acc_per_type': acc_array.tolist(),
        'PQD': PQD,
        'DPM': DPM,
        'EOM': EOM
    }

    if is_binary:
        fpr, tpr, _ = roc_curve(global_labels, positive_list)
        AUC = auc(fpr, tpr)
        result['AUC'] = AUC

        aucs, tprs, fprs = [], [], []
        for probs, labels in zip(group_probs, group_labels):
            if len(set(labels)) < 2:
                aucs.append(None)
                tprs.append(None)
                fprs.append(None)
                continue
            fpr_g, tpr_g, _ = roc_curve(labels, probs)
            auc_g = auc(fpr_g, tpr_g)
            aucs.append(auc_g)
            tprs.append(tpr_g[1])
            fprs.append(fpr_g[1])

        valid_aucs = [a for a in aucs if a is not None]
        result['min_auc'] = min(valid_aucs) if valid_aucs else None
        result['max_auc'] = max(valid_aucs) if valid_aucs else None

        tprs_valid = [t for t in tprs if t is not None]
        fprs_valid = [f for f in fprs if f is not None]
        if tprs_valid and fprs_valid:
            eod = max(abs(max(tprs_valid) - min(tprs_valid)),
                      abs(max(fprs_valid) - min(fprs_valid)))
        else:
            eod = None
        result['EOD'] = eod

    return result


In [1]:
#usage of above binary sensitive attribute 
#results = cal_metrics(" ", is_binary=True)
#print(results)